In [8]:
# --installing a few stuffs first--
# pip install pandas
# pip install requests
# pip install bs4
# pip install geopy
# pip install selenium
# for installing selenium webdriver, see these 2 youtube tutorials for references:
# https://bit.ly/3D0InrZ
# https://bit.ly/3wnvGWS
# --importing pandas--
import os
import pandas
# --importing web scraping stuff--
import requests
from bs4 import BeautifulSoup
# --importing geopy stuff--
from geopy.geocoders import ArcGIS
# --importing selenium webdriver for opening and navigating through webpages--
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
# --importing time for making action delays (if needed)--
import time

# --stuff for geopy--
nom = ArcGIS()
# --creating a new empty dataframe with just columns
df = pandas.DataFrame(columns=['Street Address','City','State','ZIP Code','Location','Coordinates','Latitude','Longitude'])
# --disabling browser popup notifications--
option1 = Options()
option1.add_argument("--disable-notifications")
# --locating chromedriver in directory--
driver = webdriver.Chrome(executable_path = 'C:\\Users\\mdima\\Desktop\\Python Learning Files\\02\\chromedriver.exe', options = option1)
# --opening browser and the homepage--
driver.maximize_window()
driver.get("https://locations.traderjoes.com")
# --defining index for later iteration--
index200 = list(range(0,200))
# --empty list to save multiple xpath from the page--
xpath_list = []
# --making a list of clickable xpaths--
for item in index200:
    xpath = '/html/body/div[2]/div[2]/main/div/div/div/div[2]/div[4]/div/div/div/div[2]/div/div[%s]/a'%str(index200.index(item)+1)
    xpath_list.append(xpath)
# --empty list to save all the stores' datas (address, zip code etc.)--
l = []
# --empty list to store all urls of the cities--
stores = []
for item in xpath_list:
    try:
        # --finding and clicking every states in the homepage--
        # --moving to city page after clicking--
        # --will stop finding and clicking once theres no more clickable states--
        item2 = xpath_list[0]
        link = driver.find_element(By.XPATH, item)
        link.click()
        for item2 in xpath_list:     
            try:
                # --finding and clicking every cities in the state page--
                # --moving to stores page after clicking--
                # --will stop finding and clicking once theres no more clickable cities--
                link2 = driver.find_element(By.XPATH, item2)
                link2.click()
                # --getting the url of the clicked city, which has all infos for the stores--
                current = driver.current_url
                # --the urls will then be saved in the list for further use--
                stores.append(current)
                # --not much more to do here, so we go back one page from store page to city page--
                driver.back()
            except:
                # --when theres no more clickable city, then go back one page from city page to homepage--
                driver.back()
                break
    except:
        break
# --now we got the urls from each city. each url will be soup'ed--
for store in stores:
    # --beautifulsoup stuff--
    r = requests.get(store)
    c = r.content
    soup = BeautifulSoup (c, "html.parser")
    # --in the store page, inspect the block with the store infos in it--
    # --it has a "div" tag with class "address-left"--
    all = soup.find_all("div",{"class":"address-left"})
    for i in all:
        d = {}
        # --web scraping stuff with BeautifulSoup--
        Address = i.find_all("span")[1].text
        City = i.find_all("span")[2].text
        State_l = soup.find("div",{"class":"breadcrumb container"}).find_all("a")[2].text
        State_s = i.find_all("span")[3].text
        Zipcode = i.find_all("span")[4].text
        Country = i.find_all("span")[5].text
        # --to obtain latitude and longitude, we need this arrangement for the data--
        Location = Address+", "+City+", "+State_s+" "+Zipcode+", "+Country
        # --d is a dictionary. so we add values to them keys--
        # --d["key"] = value--
        d["Street Address"] = Address
        d["City"] = City
        d["State"] = State_l
        d["ZIP Code"] = Zipcode
        d["Location"] = Location
        # --latitude and longitude function can only be used once the data has been transformed in dataframe form--
        # --adding the elements of the dictionary to the previously prepared empty list--
        # --because we need a list to make a dataframe--
        l.append(d)
# --now transferring all the data from the list l to the empty dataframe df we made earlier--       
df = pandas.DataFrame(l)
# --geopy functions work only with dataframes (probably)--
# --turning the location into coordinates
df["Coordinates"] = df["Location"].apply(nom.geocode)
# --extracting latitude values from the coordinates--
df["Latitude"] = df["Coordinates"].apply(lambda x: x.latitude if x != None else None)
# --extracting longitude values from the coordinates--
df["Longitude"] = df["Coordinates"].apply(lambda x: x.longitude if x != None else None)
# --try printing df to see your work and be proud--
# --saving the finished dataframe to csv
df.to_csv("Trader Joes store data.csv")
# --saving the finished dataframe to excel
df.to_excel("Trader Joes store data.xlsx")

C:\Users\mdima\AppData\Local\Temp\ipykernel_872\2380066540.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = 'C:\\Users\\mdima\\Desktop\\Python Learning Files\\02\\chromedriver.exe', options = option1)
